In [1]:


import ipyparallel as ipp
c = ipp.Client()
c.ids



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print('Rank {} of {} is alive.'.format(comm.Get_rank(), comm.Get_size()))

[stdout:0] Rank 0 of 10 is alive.
[stdout:1] Rank 1 of 10 is alive.
[stdout:2] Rank 2 of 10 is alive.
[stdout:3] Rank 3 of 10 is alive.
[stdout:4] Rank 4 of 10 is alive.
[stdout:5] Rank 5 of 10 is alive.
[stdout:6] Rank 6 of 10 is alive.
[stdout:7] Rank 7 of 10 is alive.
[stdout:8] Rank 8 of 10 is alive.
[stdout:9] Rank 9 of 10 is alive.


In [3]:
%%px
lx = 20
ly = 20
d = 40

In [4]:
%%px
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
gcd = np.gcd(np.arange(size), size)
gcd_sorted = np.sort(gcd)
y_size = gcd_sorted[-2]
x_size = size/gcd_sorted[-2]
cartesian2d = comm.Create_cart(dims = [x_size, y_size], periods =[True, True],reorder=False)
rank = comm.Get_rank()
coord2d = cartesian2d.Get_coords(rank)
print ("In 3D topology, Processor ",rank, " has coordinates ",coord2d)

[stdout:0] In 3D topology, Processor  0  has coordinates  [0, 0]
[stdout:1] In 3D topology, Processor  1  has coordinates  [0, 1]
[stdout:2] In 3D topology, Processor  2  has coordinates  [0, 2]
[stdout:3] In 3D topology, Processor  3  has coordinates  [0, 3]
[stdout:4] In 3D topology, Processor  4  has coordinates  [0, 4]
[stdout:5] In 3D topology, Processor  5  has coordinates  [1, 0]
[stdout:6] In 3D topology, Processor  6  has coordinates  [1, 1]
[stdout:7] In 3D topology, Processor  7  has coordinates  [1, 2]
[stdout:8] In 3D topology, Processor  8  has coordinates  [1, 3]
[stdout:9] In 3D topology, Processor  9  has coordinates  [1, 4]


In [5]:
%%px
n_local_x = lx // x_size
n_local_y = ly // y_size
if coord2d[0] + 1 == x_size:
    n_local_x = lx - n_local_x * (x_size - 1)
    
if coord2d[1] + 1 == y_size:
    n_local_y = ly - n_local_y * (y_size - 1)
    
#if coords2d[0] + 1 <= lx - n_local_x * x_size:
#    n_local_x += 1

#if coords2d[1] + 1 <= ly - n_local_y * y_size:
#    n_local_y += 1
print(n_local_x, n_local_y)

[stdout:0] 10.0 4
[stdout:1] 10.0 4
[stdout:2] 10.0 4
[stdout:3] 10.0 4
[stdout:4] 10.0 4
[stdout:5] 10.0 4
[stdout:6] 10.0 4
[stdout:7] 10.0 4
[stdout:8] 10.0 4
[stdout:9] 10.0 4


In [6]:
%%px
left, right = cartesian2d.Shift(direction = 0,disp=1)
print('Process ',rank, ' coordinates ', coord2d, ' with neighbors in x', left, right)

bottom, top = cartesian2d.Shift(direction = 1,disp=1)
print('Process ',rank, ' coordinates ', coord2d, ' with neighbors in y ', top, bottom)

[stdout:0] 
Process  0  coordinates  [0, 0]  with neighbors in x 5 5
Process  0  coordinates  [0, 0]  with neighbors in y  1 4
[stdout:1] 
Process  1  coordinates  [0, 1]  with neighbors in x 6 6
Process  1  coordinates  [0, 1]  with neighbors in y  2 0
[stdout:2] 
Process  2  coordinates  [0, 2]  with neighbors in x 7 7
Process  2  coordinates  [0, 2]  with neighbors in y  3 1
[stdout:3] 
Process  3  coordinates  [0, 3]  with neighbors in x 8 8
Process  3  coordinates  [0, 3]  with neighbors in y  4 2
[stdout:4] 
Process  4  coordinates  [0, 4]  with neighbors in x 9 9
Process  4  coordinates  [0, 4]  with neighbors in y  0 3
[stdout:5] 
Process  5  coordinates  [1, 0]  with neighbors in x 0 0
Process  5  coordinates  [1, 0]  with neighbors in y  6 9
[stdout:6] 
Process  6  coordinates  [1, 1]  with neighbors in x 1 1
Process  6  coordinates  [1, 1]  with neighbors in y  7 5
[stdout:7] 
Process  7  coordinates  [1, 2]  with neighbors in x 2 2
Process  7  coordinates  [1, 2]  with neig

In [7]:
%%px
comm = MPI.COMM_WORLD.Create_cart((x_size, y_size), periods=(False, False))
left_src, left_dst = comm.Shift(0, -1)
right_src, right_dst = comm.Shift(0, 1)
bottom_src, bottom_dst = comm.Shift(1, -1)
top_src, top_dst = comm.Shift(1, 1)
print('Process ',rank, ' coordinates ', coord2d, ' with neighbors in y ', bottom_src, bottom_dst, top_src, top_dst)

[stdout:0] Process  0  coordinates  [0, 0]  with neighbors in y  1 -2 -2 1
[stdout:1] Process  1  coordinates  [0, 1]  with neighbors in y  2 0 0 2
[stdout:2] Process  2  coordinates  [0, 2]  with neighbors in y  3 1 1 3
[stdout:3] Process  3  coordinates  [0, 3]  with neighbors in y  4 2 2 4
[stdout:4] Process  4  coordinates  [0, 4]  with neighbors in y  -2 3 3 -2
[stdout:5] Process  5  coordinates  [1, 0]  with neighbors in y  6 -2 -2 6
[stdout:6] Process  6  coordinates  [1, 1]  with neighbors in y  7 5 5 7
[stdout:7] Process  7  coordinates  [1, 2]  with neighbors in y  8 6 6 8
[stdout:8] Process  8  coordinates  [1, 3]  with neighbors in y  9 7 7 9
[stdout:9] Process  9  coordinates  [1, 4]  with neighbors in y  -2 8 8 -2


In [37]:
%%px
if coord2d[0] == 0:
    print(coord2d)

[stdout:0] [0, 0]
[stdout:1] [0, 1]


In [38]:
%%px

n_local_x = lx // size
n_local_y = ly // size
if rank == size - 1:
    n_local_x = lx - n_local_x * (size - 1)
    n_local_y = ly - n_local_y * (size - 1)
    
a = np.ones((n_local_x, n_local_y))
print(a.shape)

[stdout:0] (105, 45)
[stdout:1] (105, 45)
[stdout:2] (105, 45)
[stdout:3] (105, 45)


In [51]:
%%px
def x_in_process(coord2d, x_coord, lx, processes_in_x):
    lower = coord2d[0] * (lx // processes_in_x)
    upper = (coord2d[0] + 1) * (lx // processes_in_x) - 1 if not coord2d[0] == processes_in_x - 1 else lx - 1
    return lower <= x_coord <= upper

def y_in_process(coord2d, y_coord, ly, processes_in_y):
    lower = coord2d[1] * (ly // processes_in_y)
    upper = (coord2d[1] + 1) * (ly // processes_in_y) - 1 if not coord2d[1] == processes_in_y - 1 else ly - 1
    return lower <= y_coord <= upper
#x_in_process(coord2d, 0, lx-1, x_size)
y_in_process(coord2d, 6, ly, y_size)

Out[0:49]: False

Out[1:49]: False

Out[2:49]: False

Out[3:49]: False

Out[4:49]: False

Out[5:49]: False

In [51]:
%%px
def test(coord2d, x_coord, lx, process_in_x):
    def f():
        if x_in_process(coord2d, x_coord, lx, process_in_x):
            print(coord2d)
    return f
test(coord2d, 0, lx, gcd_sorted[-2])()

[stdout:0] [0, 0]
[stdout:1] [0, 1]


In [60]:
%%px
def global_coord_to_local_coord(coord2d: list, global_x: int, global_y: int, lx: int, ly: int, x_size: int,
                                y_size: int):
    print(ly, ly // y_size, ly - (ly // y_size)*y_size, y_size, coord2d)
    assert x_in_process(coord2d, global_x, lx, x_size)
    assert y_in_process(coord2d, global_y, ly, y_size)
    
    local_x = global_x - coord2d[0] * (lx // x_size)
    local_y = global_y - coord2d[1] * (ly // y_size)
    return local_x, local_y
    
if coord2d == [1, 1]:
    global_coord_to_local_coord(coord2d, 5, 1, lx, ly, x_size, y_size)

[stdout:4] 
5 1 2 3 [1, 1]
2.0 0
